In [122]:
#Constraints For The Given Problem

"""
Hard Constraints (10 Points Each)
• An exam will be scheduled for each course.
• A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
• Exam will not be held on weekends.
• Each exam must be held between 9 am and 5 pm
• Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
• A teacher cannot invigilate two exams in a row.

Soft Constraints (5 Points Each)
• All students and teachers shall be given a break on Friday from 1-2.
• A student shall not give more than 1 exam consecutively.
• Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.
• If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.

Additional Constraints (Mentioned Later on, Points not known)
• 28 Students per Class
• Use binary encoding for chromosome.

"""

#Deliverables

"""
.pynb File
One Page PDF Report
"""


'\n.pynb File\nOne Page PDF Report\n'

In [123]:
# %%
# All Imports

import csv
import math
import numpy as np
from random import *

In [124]:
#Hyper Parameters

max_generations = 1000
population_size = 64
crossover_rate = 0.8
mutation_rate = 0.01

h_constraints_exp = 2
s_constraints_exp = 1
dispersion_exp = 4


#Generic Variables
max_students_per_class = 28

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
total_timeslots = len(days)*2
morning_starting = '9AM'
morning_ending = '12PM'
evening_starting = '2PM'
evening_ending = '5PM'

student_data = None
teacher_data = None
course_data = None
student_in_course = None
courses_to_student = None
timeslots = None

In [125]:
#Data Extraction from Files
def read_student_data(filename):
    #Storing student data in dictionary
    #Key is 0,1,2,3 ; value is student name
    student_data = dict()
    count = 0
    with open(filename,mode = 'r') as student_datafile:
        read = csv.reader(student_datafile,delimiter = ',')
        for name in read:
            #Bad data
            if len(name[0]) == 0:
                continue
            #studentData.add(name[0])
            student_data[count] = name[0]
            count += 1
    #print(studentData)
    return student_data
def read_teacher_data(filename):
    #Storing teacher data in dictionary
    #key is 0,1,2,3 ; value is teacher names
    teacher_data = dict()
    count = 0
    with open(filename,mode = 'r') as teacher_datafile:
        read = csv.reader(teacher_datafile,delimiter = ',')
        for name in read:
            #Bad data
            if len(name) == 0:
                continue
            teacher_data[count] = name[0]
            count += 1
    #print(teacherData)
    return teacher_data
def read_course_data(filename):
    #Storing Course data
    #Key is course code, value is course name
    courses_data = dict()
    count = 0
    with open(filename,mode = 'r') as courses_datafile:
        read = csv.reader(courses_datafile,delimiter = ',')
        for name in read:
            #Bad data
            if ( len(name[0]) == 0 or len(name[1]) == 0 ):
                continue
            courses_data[name[0]] = name[1]
    #print(coursesData)
    return courses_data
def read_student_in_course_data(filename, courses_data, student_data):
    #Storing student in each course
    #Key is course; value is list of studen taking course
    student_in_course_data = dict()
    for i in courses_data:
        student_in_course_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as student_in_course_datafile:
        read = csv.reader(student_in_course_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            #Bad data
            if name[2] not in student_in_course_data.keys():
                print("Key is " + name[2])
                user = input("Foreign key missing in read_student_in_course_data function")
            student_in_course_data[name[2]].append(student_value_list.index(name[1]))
            count += 1
    return student_in_course_data
def courses_to_student_data(filename, student_data):
    #Storing courses taken by each student
    #Key is student; value is list of courses taken by student
    courses_to_student_data = dict()
    for i in student_data:
        courses_to_student_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as courses_to_student_datafile:
        read = csv.reader(courses_to_student_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            #Bad data
            if (student_value_list.index(name[1])) not in courses_to_student_data.keys():
                user = input("Foreign key missing in courses_to_student_data function")
            courses_to_student_data[student_value_list.index(name[1])].append(name[2])
            count += 1
    #print(courses_to_student_data)
    return courses_to_student_data

def create_timeslots():
    global timeslots
    global days
    global total_timeslots
    global morning_starting
    global morning_ending
    global evening_starting
    global evening_ending
    
    timeslots = dict()
    for i in range(total_timeslots):
        start_time = None
        end_time = None
        if i%2 == 0:
            start_time = morning_starting
            end_time = morning_ending
        else:
            start_time = evening_starting
            end_time = evening_ending
        timeslots[i] = [start_time, end_time, days[int(i/2)]]

In [126]:
#Miscellaneous Functions

#Calculates the Average of Students in Each Course
def avg_students_per_course():
    global student_in_course
    sum_students = 0
    leng = 0
    for i in student_in_course.keys():
        sum_students += len(student_in_course[i])
        leng += 1
    return (sum_students / leng)

#Calculates the number of teachers required for Invigilating the given Course Code (28 Students Max Per Teacher)
def teachers_needed(course_code):
    global student_in_course
    students = student_in_course[course_code]
    num_students = len(students)
    return math.ceil(num_students/max_students_per_class)

#Calculates the Number of Students Registered in the given Course Code
def students_count(course_code):
    global student_in_course
    return len(student_in_course[course_code])

#Checks whether the Given Course Code is MG or not
def is_MG(course_code):
    if course_code[0] == 'M' and course_code[1] == 'G':
        return True
    else:
        return False
    
    
#Random Debugging Print
def debug_1():
    print("")
    print(f"debug_1 Routine Called")
    print("")
    print("")
    for i in range(total_timeslots):
        print(timeslots[i])
    
    
    #Verification
    print(f"Number of Students: {len(student_data.keys())}")
    print(f"Number of Teachers: {len(teacher_data.keys())}")
    print(f"Number of Courses: {len(course_data.keys())}")
    
  
    print(f"Average Students Per Course: {avg_students}")
    print(f"Classrooms Per Course: {classrooms_per_course}")
    print(f"Average Students Per Classroom: {avg_students_per_classroom}")
    
    
    print(f"Teachers required for EE229: {teachers_needed('EE229')}")
    print(f"Students in EE229: {students_count('EE229')}")
    print(f"Teachers required for CS218: {teachers_needed('CS218')}")
    print(f"Students in CS218: {students_count('CS218')}")
    print("")
    print("")
    
def debug_2():
    print("")
    print(f"debug_2 Routine Called")
    print("")
    print("")
    #Chromosome Test
    p = chromosome()
    p.randomize()
    p.print_english_course('EE229')
    p.print_english_course('CS218')
    
    s = chromosome()
    s.randomize()
    s.print_english_course('EE229')
    s.print_english_course('CS218')
    
    child = chromosome()
    child.copy_timeslot('EE229', p)
    child.copy_teachers('EE229', p)
    child.copy_timeslot('CS218', s)
    child.copy_teachers('CS218', s)
    child.print_english_course('EE229')
    child.print_english_course('CS218')
    print("")
    print("")
    
def debug_3():
    print("")
    print(f"debug_3 Routine Called")
    print("")
    print("")
    #Testing Individual Class
    person = Individual(True)
    person.calculate_fitness()
    
    person2 = Individual(True)
    person2.calculate_fitness()
    
    child = person.crossover(person2)
    child.calculate_fitness()
    
    
    person.print_stats()
    person2.print_stats()
    child.print_stats()
    person.get_DNA().print_english()
    person2.get_DNA().print_english()
    child.get_DNA().print_english()
    print("")
    print("")

In [127]:
#Implementation of Chromosome here
class chromosome:
    #Intialize courses dictionary, course is a key, and value is 2d list, 
    #first row of list contains timeslot, second contains teachers
    def __init__(self):
        self.courses = dict()
        for key in course_data.keys():
            array_timeslots = np.zeros(total_timeslots, dtype = int)
            array_teachers = np.zeros(len(teacher_data.keys()), dtype = int)
            self.courses[key] = [array_timeslots, array_teachers]#np.vstack((array_timeslots,array_teachers)).T
    
    #Printing Chromosome Details in Binary
    def print_binary(self):
        print("-----------------------------")
        print(f"Printing Chromosome Details")
        for key in course_data.keys():
            self.print_binary_course(key)
            
    #Printing Chromosome Details of all Courses in English
    def print_english(self):
        print("-----------------------------")
        print(f"Printing Chromosome Details")
        for key in course_data.keys():
            self.print_english_course(key)
            
    #Printing Details of a Course Code in Binary
    def print_binary_course(self, course_code):
        print("--------------------")
        print(f"Course Code: {course_code}")
        print(f"Timeslots: ")
        print(f"{self.courses[course_code][0]}")
        print(f"Teachers: ")
        print(f"{self.courses[course_code][1]}")
        print("")
        
    #Printing Details of a Course Code in English
    def print_english_course(self, course_code):
        print("--------------------")
        print(f"Course Code: {course_code}")
        print(f"Timeslot: ")
        index = -1
        for i in range(total_timeslots):
            if self.courses[course_code][0][i] == 1:
                index = i
                break
        if index == -1:
            print(f"None")
        else:
            print(timeslots[index])

        index = -1
        print(f"Teacher(s): ")
        for i in range(len(teacher_data)):
            if self.courses[course_code][1][i] == 1:
                index = i
                print(teacher_data[i])
        if index == -1:
            print(f"None")
            
    #Printing Details of All Timeslots
    def print_english_2(self):
        for key in timeslots.keys():
            print("------------------------")
            print(timeslots[key])
            self.print_english_timeslot(key)
            
    #Printing Details of Each Timeslots
    def print_english_timeslot(self, timeslot):
        course_exam_existence = -1
        print(f'Scheduled Exam(s) :')
        for course_code in self.courses.keys():
            if self.courses[course_code][0][timeslot] == 1:
                course_exam_existence = 1
                print("---------------")
                print(f"Course Code: {course_code}")
                index = -1
                print(f"Teacher(s): ")
                for i in range(len(teacher_data)):
                    if self.courses[course_code][1][i] == 1:
                        index = i
                        print(teacher_data[i])
                if index == -1:
                    print(f"None")
        if course_exam_existence == -1:
              print(f"None")
        
            
    #Get element from timeslot of teacher        
    def get_value(self,course_code, array, element):
        return self.courses[course_code][array][element]
    
    #Set element of timeslot or teacher
    def set_value(self,course_code, array, element, value):
        self.courses[course_code][array][element] = value
        
    #Get Timeslot array for a course code
    def get_timeslot(self, course_code):
        return self.courses[course_code][0]
    
    #Get Teacher array for a course code
    def get_teachers(self, course_code):
        return self.courses[course_code][1]
    
    #Make Timeslot Array Zero for a Course Code
    def reset_timeslot(self, course_code):
        self.courses[course_code][0] = np.zeros(total_timeslots, dtype = int)
    
    #Make Teacher Array Zero for a Course Code
    def reset_teachers(self, course_code):
        self.courses[course_code][1] = np.zeros(len(teacher_data.keys()), dtype = int)
        
    #Copy Timeslot Array for a Course Code from Another Chromosome
    def copy_timeslot(self, course_code, dna):
        self.courses[course_code][0] = np.copy(dna.get_timeslot(course_code))
        
    #Copy Teacher Array for a Course Code from Another Chromosome
    def copy_teachers(self, course_code, dna):
        self.courses[course_code][1] = np.copy(dna.get_teachers(course_code))
    
    #Randomize Chromosome
    def randomize(self):
        for key in course_data.keys():
            self.randomize_timeslot(key)
            self.randomize_teachers(key)
                
    #Randomize Timeslot for a Course Code
    def randomize_timeslot(self, course_code):
        self.reset_timeslot(course_code)
        #Assigning a Random Timeslot
        slot_index = randint(0, total_timeslots-1)
        self.get_timeslot(course_code)[slot_index] = 1
        
    #Randomize Teachers for a Course Code
    def randomize_teachers(self, course_code):
        self.reset_teachers(course_code)
        
        #Assigning Random Teachers
        max_teachers = teachers_needed(course_code)
        count = 0
        while count < max_teachers:
            teacher_index = randint(0, len(teacher_data.keys())-1)
            while self.get_teachers(course_code)[teacher_index] != 0:
                teacher_index = randint(0, len(teacher_data.keys())-1)

            self.get_teachers(course_code)[teacher_index] = 1
            count += 1
        

In [128]:
#Implementation of Individual Class
class Individual:
    def __init__(self, is_random):
        self.DNA = chromosome()
        
        self.weight_dispersion = -1
        self.weight_h_constraints = -1
        self.weight_s_constraints = -1
        
        self.student_conflicts_a = -1
        self.student_conflicts_b = -1
        self.teacher_conflicts_a = -1
        self.teacher_conflicts_b = -1
        self.courses_conflicts = -1
        
        self.fitness = 0
        
        if is_random == True:
            self.DNA.randomize()
    
    def get_DNA(self):
        return self.DNA
    
    def get_fitness(self):
        return self.fitness
            
    def calculate_fitness(self):
        #Dispersion Weight
        self.weight_dispersion = 0
        w_sum1 = 0
        for i in range(total_timeslots):
            temp_count = 0
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    temp_count += 1
            w_sum1 += math.pow(temp_count, dispersion_exp)
            
        self.weight_dispersion = w_sum1
        
        #Hard Constraints Weight
        self.weight_h_constraints = 0
        w_sum2 = 0
        #Checking Students Giving more than 1 exam at same time
        temp_count = 0
        for i in range(total_timeslots):
            cur_courses = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    cur_courses.append(j)
            checked = []
            for j in cur_courses:
                for k in cur_courses:
                    if k == j or k in checked:
                        continue
                    for student in student_in_course[j]:
                        if student in student_in_course[k]:
                            temp_count += 1
                checked.append(j)
                            
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.student_conflicts_a = temp_count
        #Checking if Teacher is invigilating more than 1 exams at same time
        temp_count = 0
        for i in range(total_timeslots):
            #cur_courses = []
            assigned_teachers = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    #cur_courses.append(j)
                    temp_teachers = []
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1:
                            temp_teachers.append(k)
                    assigned_teachers.append(temp_teachers)
            for j in range(len(assigned_teachers)):
                for k in range(j+1, len(assigned_teachers)):
                    for teacher in assigned_teachers[j]:
                        if teacher in assigned_teachers[k]:
                            temp_count += 1
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.teacher_conflicts_a = temp_count
        #Checking if Teacher us invigilating two exams in a row
        temp_count = 0
        for i in range(0, total_timeslots, 2):
            cur_teachers = []
            next_teachers = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1 and k not in cur_teachers:
                            cur_teachers.append(k)
                if self.DNA.get_timeslot(j)[i+1] == 1:
                    for k in range(len(teacher_data.keys())):
                        if self.DNA.get_teachers(j)[k] == 1 and k not in next_teachers:
                            next_teachers.append(k)
            for j in cur_teachers:
                if j in next_teachers:
                    temp_count += 1
        w_sum2 += math.pow(temp_count, h_constraints_exp)
        self.teacher_conflicts_b = temp_count

        
        #Finalizing Hard Constraints Weight
        self.weight_h_constraints = w_sum2
        
        #Soft Constraints Weight
        self.weight_s_constraints = 0
        w_sum3 = 0
        #Checking Students Giving more than 1 exam in a day
        temp_count = 0
        for i in range(0, total_timeslots, 2):
            cur_students = []
            next_students = []
            for j in course_data.keys():
                if self.DNA.get_timeslot(j)[i] == 1:
                    for k in student_in_course[j]:
                        if k in cur_students:
                            continue
                        cur_students.append(k)
                if self.DNA.get_timeslot(j)[i+1] == 1:
                    for k in student_in_course[j]:
                        if k in next_students:
                            continue
                        next_students.append(k)
            for j in cur_students:
                if j in next_students:
                    temp_count += 1
        w_sum3 += math.pow(temp_count, s_constraints_exp)
        self.student_conflicts_b = temp_count
        #Checking if CS exam of Student Comes Before MG
        temp_count = 0
        for i in student_data.keys():
            mg_courses = []
            others = []
            for j in courses_to_student[i]:
                if is_MG(j):
                    mg_courses.append(j)
                else:
                    others.append(j)
            for j in mg_courses:
                index1 = -1
                index2 = -1
                for k in self.DNA.get_timeslot(j):
                    index1 += 1
                    if k == 1:
                        break
                for k in others:
                    for l in self.DNA.get_timeslot(k):
                        index2 += 1
                        if l == 1:
                            break
                    
                    if index1 >= index2:
                        temp_count += 1
        w_sum3 += math.pow(temp_count, s_constraints_exp)
        self.courses_conflicts = temp_count

        
        #Finalizing Soft Constraints Weight
        self.weight_s_constraints = w_sum3
            
        
        #Fitness
        self.fitness = 100000/(self.weight_dispersion+self.weight_h_constraints+self.weight_s_constraints + 1)
        return self.fitness
    
    #Check if this individual is Perfect (H+S Satisfied)
    def is_perfect(self):
        if self.weight_h_constraints == 0:
            if self.weight_s_constraints == 0:
                return True
        return False
    
    #Check if this individual is Semi-Perfect (H Satisfied)
    def is_semiperfect(self):
        if self.weight_h_constraints == 0:
            return True
        return False
    
    def crossover(self, other_parent):
        child = Individual(False)
        
        #Passing on Self DNA
        for i in range(0, len(course_data.keys()), 2):
            key = None
            index1 = -1
            #Getting Course Code for the Given Index
            for j in course_data.keys():
                index1 += 1
                if index1 == i:
                    key = j
                    break
            
            #Checking if to Mutate or Not
            mutate_proc = randint(0, 100)
            if mutate_proc < mutation_rate*100: #Mutate
                child.get_DNA().randomize_timeslot(key)
                child.get_DNA().randomize_teachers(key)
            else: #No Mutation
                child.get_DNA().copy_timeslot(key, self.DNA)
                child.get_DNA().copy_teachers(key, self.DNA)
                
        
        #Passing on Other Parent DNA
        for i in range(1, len(course_data.keys()), 2):
            key = None
            index1 = -1
            #Getting Course Code for the Given Index
            for j in course_data.keys():
                index1 += 1
                if index1 == i:
                    key = j
                    break
            
            #Checking if to Mutate or Not
            mutate_proc = randint(0, 100)
            if mutate_proc < mutation_rate*100: #Mutate
                child.get_DNA().randomize_timeslot(key)
                child.get_DNA().randomize_teachers(key)
            else: #No Mutation
                child.get_DNA().copy_timeslot(key, other_parent.get_DNA())
                child.get_DNA().copy_teachers(key, other_parent.get_DNA())
        
        
        #Returning Child
        return child
        
    def write_to_csv(self, filename):
        with open(filename, 'w') as file:
            writer = csv.writer(file)
            writer.writerow(["","9AM - 12PM","2PM - 5PM"])
            current_exams_morning = ""
            current_exams_afternoon = ""
            #Iterate every single timeslot
            for i in range(0,len(timeslots)):
                if ( i % 2 == 0 ):
                    #iterate every course in each timeslot
                    for key in self.DNA.courses.keys():
                        #Check if ith time slot index is 1 aganist that course
                        get_timeslot = self.DNA.get_timeslot(key)
                        if get_timeslot[i] == 1:
                            if len(current_exams_morning) == 0:
                                current_exams_morning = key
                            else:
                                current_exams_morning = current_exams_morning + "/" + key
                else:
                    #iterate every course in each timeslot
                    for key in self.DNA.courses.keys():
                        #Check if ith time slot index is 1 aganist that course
                        get_timeslot = self.DNA.get_timeslot(key)
                        if get_timeslot[i] == 1:
                            if len(current_exams_afternoon) == 0:
                                current_exams_afternoon = key
                            else:
                                current_exams_afternoon = current_exams_afternoon + "/" + key
                    writer.writerow([days[i // 2],current_exams_morning,current_exams_afternoon])
                    current_exams_morning = ""
                    current_exams_afternoon = ""
                #print(f"Exam on time slot {i} is : {current_exams_afternoon}")
                    
                
    def print_stats(self):
        print("------------")
        print(f"Individual Stats:")
        print(f"Fitness:{round(self.fitness, 2)} D:{self.weight_dispersion} H:{round(self.weight_h_constraints, 2)} S:{round(self.weight_s_constraints, 2)}")
        print(f"[H] Student Conflicts (Multiple Exams Same Time):{self.student_conflicts_a}")
        print(f"[S] Student Conflicts (Multiple Exams Same Day):{self.student_conflicts_b}")
        print(f"[H] Teacher Conflicts (Same Teacher Multiple Exams):{self.teacher_conflicts_a}")
        print(f"[H] Teacher Conflicts (Same Teacher Twice in a Row):{self.teacher_conflicts_b}")
        print(f"[S] Courses Conflicts (MG Course exams after others):{self.courses_conflicts}")
    

In [129]:
#Implementation of Population Class
class Population:
    def __init__(self):
        self.population = []
        self.perfect = []
        self.semiperfect = []
        self.best = None
        self.max_fitness = 0
        self.avg_fitness = 0
        for i in range(population_size):
            self.population.append(Individual(True))
    
        self.calculate_fitness()
    
    def calculate_fitness(self):
        self.avg_fitness = 0
        for person in self.population:
            value = person.get_fitness()
            if value <= 0:
                value = person.calculate_fitness()
            self.avg_fitness += value
            if value > self.max_fitness:
                self.max_fitness = value
                self.best = person
            if person.is_perfect():
                self.perfect.append(person)
            elif person.is_semiperfect():
                self.semiperfect.append(person)
        self.avg_fitness = self.avg_fitness/population_size
                
    def selection(self):
        #Selecting Parent A
        pick_proc = randint(0, int(self.max_fitness))
        parent_a = self.population[randint(0, population_size-1)]
        while parent_a.get_fitness() < pick_proc:
            parent_a = self.population[randint(0, population_size-1)]
            
        #Selecting Parent B
        pick_proc = randint(0, int(self.max_fitness))
        parent_b = self.population[randint(0, population_size-1)]        
        while parent_b.get_fitness() < pick_proc:
            parent_b = self.population[randint(0, population_size-1)]
        
        return parent_a, parent_b
    def selection2(self):
        if population_size <= 1:
            print(f"In selection2 function, population size <= 1, which means we can't select parent b, Fix me please daddy!")
        parent_a = self.population[0]
        parent_b = self.population[0]
        for i in range (1,population_size):
            if ( self.population[i].get_fitness() > parent_a.get_fitness() ):
                parent_b = parent_a
                parent_a = self.population[i]
            elif ( self.population[i].get_fitness() > parent_b.get_fitness() ):
                parent_b = self.population[i]
        return parent_a, parent_b
    def proceed_generation(self):
        new_population = []
        
        while len(new_population) < population_size:
            #print(f"Size: {len(new_population)}")
            crossover_proc = randint(0, 100)
            if crossover_proc <= crossover_rate*100: #Crossover
#                 child = None
#                 while(True):
#                     #Selecting Parents
#                     parent_a, parent_b = self.selection()

#                     #Crossover of Parents
#                     child = parent_a.crossover(parent_b)
                    
#                     #If Child is Good Enough
#                     value = child.calculate_fitness()
#                     print(f"{round(value, 2)}/{round(self.av_fitness, 2)}")
#                     if value > self.avg_fitness:
#                         break
                #Selecting Parents
                parent_a, parent_b = self.selection2()

                #Crossover of Parents
                child_a = parent_a.crossover(parent_b)
                child_b = parent_b.crossover(parent_a)
                #Adding Child to Population
                new_population.append(child_a)
                new_population.append(child_b)

            else: #Child with Random DNA
                new_population.append(Individual(True))
                new_population.append(Individual(True))
                
        self.population = new_population
                
                
    def genetic_algorithm(self):
        print(f"Starting Genetic Algorithm")
        #Main Loop for Generations
        for i in range(max_generations):
            #Print Stats Every 100th Generation
            if i%1 == 0:
                print(f"Generation {i}/{max_generations}")
                self.best_stats()
                self.avg_stats()
                #self.solution_stats()
                
                
            #Implementation Here
            #Generating New Population
            self.proceed_generation()
            
            #Calculate New Fitness
            self.calculate_fitness()
            
            if len(self.perfect) != 0:
                print("Perfect Solution Found!")
                break

        
        self.print_solutions()
        self.best.write_to_csv("Output.csv")
        
            
                

    def solution_stats(self):
        print(f"------SOLUTIONS------")
        print(f"Perfect Solutions: {len(self.perfect)}")
        print(f"Semi-Perfect Solutions: {len(self.semiperfect)}")
    def avg_stats(self):
        print(f"------AVG------")
        print(f"Average Fitness: {round(self.avg_fitness, 2)}")
    def best_stats(self):
        print(f"------BEST------")
        self.best.print_stats()
    def print_best_binary(self):
        self.best.get_DNA().print_binary()
    def print_best_english(self):
        self.best.get_DNA().print_english()
    def print_best_english_2(self):
        self.best.get_DNA().print_english_2()
    def print_solutions(self):
        print(f"Perfect Solutions-------")
        for i in self.perfect:
            i.print_stats()
        if len(self.perfect) == 0:
            print("None")
        print(f"Semi Perfect Solutions-------")
        for i in self.semiperfect:
            i.print_stats()
        if len(self.semiperfect) == 0:
            print("None")
        
    

In [ ]:
if __name__ == "__main__":
    global student_data
    global teacher_data
    global course_data
    global student_in_course
    global courses_to_student
    global timeslots
    
    #Initialization
    student_data = read_student_data('cStudent.csv')
    teacher_data = read_teacher_data('cTeachers.csv')
    course_data = read_course_data('cCourses.csv')
    student_in_course = read_student_in_course_data('cCoursesStudent.csv', course_data, student_data)
    courses_to_student = courses_to_student_data('cCoursesStudent.csv', student_data)
    #student_data = read_student_data('studentNames.csv')
    #teacher_data = read_teacher_data('teachers.csv')
    #course_data = read_course_data('courses.csv')
    #student_in_course = read_student_in_course_data('studentCourse.csv', course_data, student_data)
    #courses_to_student = courses_to_student_data('studentCourse.csv', student_data)
    
    avg_students = avg_students_per_course()
    classrooms_per_course = math.ceil(len(teacher_data.keys())/len(course_data.keys()))
    avg_students_per_classroom = math.ceil(avg_students/classrooms_per_course)
    
    create_timeslots()
    
    #Testing Population
    population = Population()
    population.genetic_algorithm()
    
    
    

Starting Genetic Algorithm
Generation 0/1000
------BEST------
------------
Individual Stats:
Fitness:833.33 D:69.0 H:16.0 S:34.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] Student Conflicts (Multiple Exams Same Day):21
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):13
------AVG------
Average Fitness: 171.69
Generation 1/1000
------BEST------
------------
Individual Stats:
Fitness:833.33 D:69.0 H:16.0 S:34.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] Student Conflicts (Multiple Exams Same Day):21
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):13
------AVG------
Average Fitness: 284.64
Generation 2/1000
------BEST------
------------
Individual Stats:
Fitness:833.33 D:69.0 H:16.0 S:34.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] 

Generation 22/1000
------BEST------
------------
Individual Stats:
Fitness:934.58 D:55.0 H:4.0 S:47.0
[H] Student Conflicts (Multiple Exams Same Time):2
[S] Student Conflicts (Multiple Exams Same Day):23
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):24
------AVG------
Average Fitness: 809.92
Generation 23/1000
------BEST------
------------
Individual Stats:
Fitness:934.58 D:55.0 H:4.0 S:47.0
[H] Student Conflicts (Multiple Exams Same Time):2
[S] Student Conflicts (Multiple Exams Same Day):23
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):24
------AVG------
Average Fitness: 723.03
Generation 24/1000
------BEST------
------------
Individual Stats:
Fitness:934.58 D:55.0 H:4.0 S:47.0
[H] Student Conflicts (Multiple Exams Same Time):2
[S] Student Conflicts (Multiple

Generation 44/1000
------BEST------
------------
Individual Stats:
Fitness:943.4 D:55.0 H:9.0 S:41.0
[H] Student Conflicts (Multiple Exams Same Time):3
[S] Student Conflicts (Multiple Exams Same Day):20
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):21
------AVG------
Average Fitness: 657.94
Generation 45/1000
------BEST------
------------
Individual Stats:
Fitness:943.4 D:55.0 H:9.0 S:41.0
[H] Student Conflicts (Multiple Exams Same Time):3
[S] Student Conflicts (Multiple Exams Same Day):20
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):21
------AVG------
Average Fitness: 852.14
Generation 46/1000
------BEST------
------------
Individual Stats:
Fitness:943.4 D:55.0 H:9.0 S:41.0
[H] Student Conflicts (Multiple Exams Same Time):3
[S] Student Conflicts (Multiple Ex

Generation 66/1000
------BEST------
------------
Individual Stats:
Fitness:1000.0 D:55.0 H:17.0 S:27.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):1
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):5
------AVG------
Average Fitness: 732.4
Generation 67/1000
------BEST------
------------
Individual Stats:
Fitness:1000.0 D:55.0 H:17.0 S:27.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):1
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):5
------AVG------
Average Fitness: 782.1
Generation 68/1000
------BEST------
------------
Individual Stats:
Fitness:1000.0 D:55.0 H:17.0 S:27.0
[H] Student Conflicts (Multiple Exams Same Time):4
[S] Student Conflicts (Multiple 

Generation 88/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1102.56
Generation 89/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1035.26
Generation 90/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple

...

Generation 262/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 778.42
Generation 263/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1040.89
Generation 264/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multip

Generation 284/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1020.28
Generation 285/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1091.56
Generation 286/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multi

Generation 306/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 936.4
Generation 307/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1045.59
Generation 308/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multipl

Generation 329/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1114.56
Generation 330/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1079.01
Generation 331/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multi

Generation 350/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 988.03
Generation 351/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multiple Exams Same Day):22
[H] Teacher Conflicts (Same Teacher Multiple Exams):0
[H] Teacher Conflicts (Same Teacher Twice in a Row):0
[S] Courses Conflicts (MG Course exams after others):1
------AVG------
Average Fitness: 1010.17
Generation 352/1000
------BEST------
------------
Individual Stats:
Fitness:1250.0 D:55.0 H:1.0 S:23.0
[H] Student Conflicts (Multiple Exams Same Time):1
[S] Student Conflicts (Multip